In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    ConfusionMatrixDisplay,
)

In [ ]:
# Load model
import pickle


def load_model(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)


model = load_model(filename="../model/model.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '../model/model.pkl'

In [7]:
# Load completely transformed data with best sellers added
file_path = "../data/processed/vehicle_sales_model_ready_with_bestseller.parquet"
df = pd.read_parquet(file_path)

# Rename monthly_best_seller to monthly_best_seller_actual
df = df.rename(columns={"monthly_best_seller": "monthly_best_seller_actual"})

print(df.columns)
df.head(3)

Index(['price', 'mileage_km_total', 'crash_test_score', 'body_style', 'color',
       'transmission', 'dealer_region', 'brand_model', 'engine', 'year',
       'month', 'current_month_sales', 'last_1_months_sales',
       'last_2_months_sales', 'last_3_months_sales', 'last_4_months_sales',
       'last_5_months_sales', 'last_6_months_sales', 'last_7_months_sales',
       'last_8_months_sales', 'last_9_months_sales', 'last_10_months_sales',
       'last_11_months_sales', 'last_12_months_sales',
       'monthly_best_seller_actual'],
      dtype='object')


,price,mileage_km_total,crash_test_score,body_style,color,transmission,dealer_region,brand_model,engine,year,...,last_4_months_sales,last_5_months_sales,last_6_months_sales,last_7_months_sales,last_8_months_sales,last_9_months_sales,last_10_months_sales,last_11_months_sales,last_12_months_sales,monthly_best_seller_actual
0,26000,68468,4,SUV,Black,Auto,Middletown,Ford-Expedition,Double Overhead Camshaft,2022,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0
1,19000,64053,3,SUV,Black,Auto,Aurora,Dodge-Durango,Double Overhead Camshaft,2022,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0
2,31500,24942,5,Passenger,Red,Manual,Greenville,Cadillac-Eldorado,Overhead Camshaft,2022,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,1


In [ ]:
# Get test data (the one used for models evaluation that models were not trained on)
test_cond = (df["year"] == 2023) & (df["month"] >= 10)
test_df = df[test_cond]
X_test = test_df.drop(columns=["monthly_best_seller_actual"])
y_test = test_df["monthly_best_seller_actual"]

# Make predictions and add them as a new column
test_df["monthly_best_seller_predicted"] = model.predict(X_test)

test_df.head(3)